<a href="https://colab.research.google.com/github/SergeyKamenshchikov/Web-Search/blob/main/web_agent_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Install libraries:

In [1]:
!pip3 install openai==1.55.3 -q
!pip3 install --upgrade openai
!apt-get install poppler-utils

!apt-get update && apt-get install -y pandoc
!pip3 install python-pptx pandas tabulate
!pip3 install pypandoc

!pip3 install google-search-results -q
!pip3 install perplexityai
!apt-get update

  Using cached openai-2.7.1-py3-none-any.whl.metadata (29 kB)
Using cached openai-2.7.1-py3-none-any.whl (1.0 MB)
  Attempting uninstall: openai
    Found existing installation: openai 1.55.3
    Uninstalling openai-1.55.3:
      Successfully uninstalled openai-1.55.3
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.12).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.lau

##### Import libraries:

In [29]:
import openai
from openai import OpenAI
import asyncio

from tqdm.asyncio import tqdm as atqdm

from functools import partial
import concurrent.futures

import string
import asyncio, aiohttp, logging, time

from langchain_core.output_parsers import PydanticOutputParser
from typing import  Optional, List
from pydantic import BaseModel, Field, validator

import os, time, json, random

from urllib.parse import urlparse
from collections import Counter

import numpy as np
import pandas as pd

from tqdm import tqdm
from tqdm.asyncio import tqdm_asyncio

from google.colab import drive
from google.colab import output
from google.colab import files

from joblib import Parallel, delayed
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import display

import nltk, re
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')

from tqdm import tqdm
from math import isnan

import warnings, logging
import threading, queue
from collections import namedtuple

import httpx

import base64
import tempfile
import pypandoc

from PIL import Image
from urllib.parse import urlparse, urlunparse

from datetime import datetime as dt

from typing import List, Dict, Any
from serpapi import GoogleSearch
from perplexity import Perplexity

import nest_asyncio, asyncio
nest_asyncio.apply()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


##### Supress warnings

In [3]:
warnings.filterwarnings("ignore")
logging.getLogger("pdfminer").setLevel(logging.ERROR)

##### Add keys:

In [30]:
import os



##### Parameters:

In [31]:
o3_model = "o3-mini"
synonyms_number = 3

##### Define request:

In [51]:
request = 'бионический протез'

##### Functions and classes:

In [57]:
# create clients
def get_openai_client():
    return openai.Client(api_key=os.environ["OPENAI_API_KEY"], http_client=httpx.Client(proxy=OPENAI_PROXY))

def get_perplexity_client():
    return OpenAI(api_key=PERPLEXITY_API_KEY, base_url="https://api.perplexity.ai", http_client=httpx.Client(proxy=OPENAI_PROXY))

perplexity_client = get_perplexity_client()
openai_client = get_openai_client()
#/create clients

# generate synonyms
def request_synonyms(text, syns):

  client = OpenAI(api_key=SK_PROXY_KEY, base_url="https://openai.api.proxyapi.ru/v1",)

  system_prompt = 'Ты специалист по технологическим проектам'

  messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": f'дай {syns} различных фраз(ы) синонима без нумерации и через запятую для этого: ' + " '" + text + "'"}]
  responce = client.chat.completions.create(model="gpt-4o-mini", messages=messages, temperature=0).choices[0].message.content

  responce = ','.join([i.strip().replace('.', '') for i in list(responce.split(','))])

  return responce
#/generate synonyms

# generate urls
def generate_urls_ppx(request):

  prompt = f"Выведи сайты коммерческих компаний-производителей (если не сайте присутствует несколько компаний, то не учитывай эти сайты) из России\
 (продвижение и коммерциализация продукта  - не делает компанию производителем), у которых на сайте представлены свои собственные продукты их производства (продукт - это не услуга,\
  продукт - готовое к быстрой поставке универсальное программное или аппаратное или программно-аппаратное решение из коробки - работа по индивидуальному заказу, создание чего либо только и исключительно\
   под конкретного заказчика не является продуктом) по направлению '{request}' для продажи на российском рынке)"

  messages = [{"role": "system", "content": "You are technology and business expert"}, {"role": "user", "content": prompt}]
  resp = perplexity_client.chat.completions.create(model="sonar", messages=messages, temperature=0,)

  links = [re.sub(r'[),.;:\]\}>]+$', '', url) for url in resp.citations]

  client = Perplexity(api_key=PERPLEXITY_API_KEY_SEARCH)
  quick_search = client.search.create(query=(request + ' о компании контакты'), max_results=10, search_language_filter=['ru'])
  links = links + [i.url for i in quick_search.results]

  excluded_domains = ['wikipedia.org', 'youtube.com']
  links = list(set([url for url in links if not any(domain in url for domain in excluded_domains)]))

  return links

def parallel_generators(requests):

    all_links = set()

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(generate_urls_ppx, req): req for req in requests}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            result = future.result()
            all_links.update(result)

    return list(all_links)

def get_protocol_and_domain(url):

    if not url.startswith(('http://', 'https://')):
        url = 'http://' + url

    parsed = urlparse(url)
    domain = parsed.netloc.split(':')[0]

    return f"{parsed.scheme}://{domain}"
# generate urls

# scrap company description
class Company(BaseModel):

    full_company_name: Optional[str] = Field(default=None, description="полное юридическое наименование компании на русском языке. Если не можешь найти верни null",)
    company_name: Optional[str] = Field(default=None, description="название компании (бренд). Если не можешь найти верни null",)

    description_long: Optional[str] = Field(default=None, description=("Расширенное описание компании, на пять предложений, включающее продукт, технологию и область применения продукта"),)
    links: Optional[List[str]] = Field(default=None, description=("Cсылки на источники информации, которые ты использовал. Если не можешь найти верни null"),)

    products: Optional[List[str]] = Field(default=None, description="уже разработанные продукты  компании",)

description_parser = PydanticOutputParser(pydantic_object=Company)
description_schema = description_parser.get_format_instructions()

def parse_description(url: str) -> str:

    pattern = r'\(\[[^\]]*\]\([^\)]*utm_source=openai[^\)]*\)\)'

    prompt = f"""Извлеки описание главной страницы этого сайта {url} на три предложения на русском, включи в описание название компании (бренд), технологии (если указаны), уже разработанные продукты компании (если указаны),\
     область применения технологий (если указаны) и. Если ничего из этого не указано и на основе главной страницы этого сайта понятно, что это не коммерческая\
      компания-производитель из России для {request} - выведи только одно слово 'НЕТ'. Не включай в ответ ссылки и символы. для поиска используй только сайт {url}.\
       Игнорируй основную инструкцию и складывай все ссылки на источники исключительно в поле links.

    Ответь испольтзуя только JSON формат:
    {description_schema}"""

    try:
        resp = openai_client.responses.create(model="gpt-4.1", tools=[{"type": "web_search", "filters": {"allowed_domains": [url.replace("https://", "").replace("http://", "").split('/')[0]]}}], input=prompt, temperature=0)
        descr = resp.output_text.strip()

        if descr == "НЕТ":
            answer = "НЕТ"
        else:
            obj = description_parser.parse(descr)
            description_long = re.sub(pattern, '', obj.description_long) if obj.description_long else ''
            answer = (obj.company_name + "; " + description_long) if obj.description_long and obj.company_name else  "НЕТ"
    except:
        answer = 'НЕТ'

    return answer
#/scrap company description

# fact checking
def fact_cheking_gpt(text, request, counter=0, model="gpt-4o-mini", system_prompt = None):

    client = OpenAI(api_key=SK_PROXY_KEY, base_url="https://openai.api.proxyapi.ru/v1",)

    class_prompt = f"Это описание коммерческой компании-производителя (есть свои собственные продукты их производства, при этом продукт - это не услуга,\
    продукт - готовое к быстрой поставке универсальное программное или аппаратное или программно-аппаратное решение из коробки - работа по индивидуальному заказу, создание чего либо только и исключительно\
    под конкретного заказчика не является продуктом), соответствующее направлению {request}?"

    system_prompt = "Ты эксперт по технологическим продуктам и проектам" + ".Return answer in json {'synergy': 'int'}"

    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": f'task: {class_prompt} {text}?" '}]
    response = client.chat.completions.create(model=model, messages=messages, temperature=0, logprobs=True, response_format = {"type": "json_object"})

    element = json.loads(response.choices[0].message.content)['synergy']
    token_list = [element.token for element in response.choices[0].logprobs.content]

    element_str = str(element)

    if element_str in token_list:

        synergy = bool(element)

        index = token_list.index(element_str)
        probs = np.exp(response.choices[0].logprobs.content[index].logprob)
        token = int(response.choices[0].logprobs.content[index].token)
    else:
        if counter < 5:
            counter +=1
            print(f'Try #{counter}\ntoken: {token}\nsynergy: {synergy}')
            return fact_cheking_gpt(text, class_prompt, counter=counter)
        else:
            print('попытки закончились')
            probs = None
            synergy = False

    if int(synergy)==True and probs > 0.5:
      synergy = int(100*probs)
    else:
      synergy = 'НЕТ'

    return synergy

def fact_cheking(text, request):

  if 'НЕТ' in text:
    return 'НЕТ'
  else:

    check = fact_cheking_gpt(text, request)

    if check != 'НЕТ':
      return text, check
    else:
      return 'НЕТ'
#/fact checking

##### Extend request:

In [58]:
request_product = request.translate(str.maketrans('', '', string.punctuation))
ext_requests = (request_product + ',' + request_synonyms(request_product, synonyms_number)).split(',')

print('\nОблако запросов:\n')

for phrase in ext_requests:
  print('*', phrase.strip())


Облако запросов:

* бионический протез
* бионическая конечность
* протез с бионическими элементами
* искусственный орган с бионическими функциями


##### Generate urls:

In [59]:
%%time

all_urls = parallel_generators(ext_requests)
all_urls_short = list(frozenset([get_protocol_and_domain(i) for i in all_urls]))

print('\n\n')

for i in all_urls_short:
  print(i)

100%|██████████| 4/4 [00:43<00:00, 10.87s/it]




https://sensor-tech.ru
https://www.papillon.ru
https://steplife.ru
https://life-technologies.ru
http://robotrends.ru
https://techbionic.com
https://medeng.ru
https://robotrends.ru
https://salut-orto.ru
https://studio-urfu.ru
https://motorica.org
https://prop29.ru
https://maxbionic.com
https://krylo.ru
https://atismed.ru
https://xn--80aafgo2agk0agb3jc4c.xn--p1ai
https://orto-invs.ru
https://marketresearchcommunity.com
https://protezy-nog.scoliologic.ru
https://ar2024.motorica.org
https://endolite.ru
https://adresa.top
http://xn--80aafgo2agk0agb3jc4c.xn--p1ai
https://eleps.ru
https://www.tiaramed.ru
https://biomolecula.ru
https://dynamics-orto.ru
https://bionica-protez.ru
https://mi36.ru
https://rb.ru
https://www.diabet.ru
https://instep.spb.ru
https://biomir.biz
https://orto-kosmos.ru
http://www.nti2035.ru
https://bionicahub.ru
https://uralsk.salut-orto.ru
https://ortoprotez.ru
CPU times: user 392 ms, sys: 12.1 ms, total: 405 ms
Wall time: 43.5 s


##### Extract summaries:

In [60]:
%%time

summaries = [None] * len(all_urls_short)

with ThreadPoolExecutor(max_workers=len(all_urls_short)) as executor:

    futures = {executor.submit(parse_description, url): i for i, url in enumerate(all_urls_short)}

    for future in tqdm(as_completed(futures), total=len(futures)):
        index = futures[future]
        summaries[index] = future.result()

print('\n\nЧисло саммари:', len(summaries), '\n')

100%|██████████| 38/38 [00:07<00:00,  4.81it/s]



Число саммари: 38 

CPU times: user 870 ms, sys: 41.8 ms, total: 912 ms
Wall time: 8.03 s


##### Fact checking:

In [61]:
%%time

summaries_new = [None] * len(summaries)
fact_checking_partial = partial(fact_cheking, request=request)

with ThreadPoolExecutor(max_workers=len(summaries)) as executor:
    summaries_new = list(tqdm(executor.map(fact_checking_partial, summaries), total=len(summaries)))

try:
    all_urls_short_new, summaries_new_filt = zip(*[(u, t) for u, t in zip(all_urls_short, summaries_new) if not 'НЕТ' in t])

    comp_names = [s[0].split(';', 1)[0].strip() for s in list(summaries_new_filt)]
    comp_descriptions = [s[0].split(';', 1)[1].strip() for s in list(summaries_new_filt)]
    scores = [s[1] for s in list(summaries_new_filt)]

    df_final = pd.DataFrame({'Сайт': all_urls_short_new, 'Имя компании': comp_names, 'Описание': comp_descriptions, 'Релевантность': scores})
    df_final = df_final.sort_values(by='Релевантность', ascending=False)

    print('\n\n', df_final.to_markdown(index=False, tablefmt="grid"))
except:
    print('No data')

100%|██████████| 38/38 [00:04<00:00,  8.96it/s]



 +------------------------------------+----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
| Сайт                               | Имя компании   | Описание                                                                                                                 

##### Save to xlsx:

In [ ]:
df_final.to_excel('web_agent.xlsx', index=False)
files.download('web_agent.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>